# Informes de Participación Somos Familia UdeA

Hola, este documento de *Google Colab* tiene como finalidad permitir realizar el procesamiento de los archivos de participantes e inscritos de reunión extraídos desde Zoom y generar un reporte de asistencia. En este caso, para la estrategia de Somos Familia UdeA, donde tenemos los siguientes archivos son:

- ***Reporte de Reunión (reg)***

- ***Reporte de Uso (par)***

Debes realizar la descarga de estos dos archivos desde el espacio de Zoom de **Mi Comunidad**. Una vez descargados, ponles un nombre reconocible que te permita discriminar la fecha y el tipo de archivo que es, como recomendación se tiene el formato de:

### reg_fecha_SFUdeA.csv

- Al inicio del archivo indica si es el Reporte de Reunión (reg) o el Reporte de Uso (par), seguido de un guion bajo ( _ ).
- A continuación, ubica la fecha del encuentro, iniciando por el mes, día y año, todo separando por guiones bajos ( _ ).
- Por último, las iniciales de la estrategia, en este caso SFUdeA.
- Ejemplo: Para el archivo de Reporte de Reunión del 9 de Agosto de 2050 el archivo resultante seria el: reg_08_09_2050_SFUdeA.csv (Recuerda que el .csv es el tipo de archivo, por lo que no lo debes cambiar)

Una vez renombrados los archivos, arrástralos hasta la sección de archivos del Google Colab ubicado en el panel izquierdo. Luego, puedes empezar con las siguientes celdas de código.

## 1. Instalación de Paquetes Necesarios

---


Para iniciar, debes correr la celda de código que se encuentra abajo. Esta celda realizara la instalación de dos paquetes necesarios para el procesamiento de los archivos.

- **Stringr**: Es un paquete que procesa texto, esto ayuda a suprimir los caracteres especiales y dejar solo en minúsculas algunas secciones necesarias de los archivos.
- **xlsx**: Es el paquete que permite generar un archivo final de Excel con todos los datos que requieras.

In [ ]:
install.packages("stringr")
install.packages("openxlsx")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘Rcpp’




## 2. Asignación de Archivos y Función de Generación

---

En la celda de abajo tenemos el codigo de la función, no es necesario que entinedas en principio como funciona, basta con que tengas claro como utilizarla.

Anterirmente cargamos los archivos **par** y **reg**, antes de correr la celda, debes modificar el nombre de cada uno en las 2 primeras lineas de codigo

```
# direccion_reg
# dirección_par
```
Es importante que los archivos se correspondan especificamente a **reg** o a **par** respectivamente, ya que de lo contrario no la función no podra realizar el proceso.

Por último, debes modificar el nombre del archivo final.

```
# nombre_arch
```

Al igual que para los otros dos archivos, es recomendable tener un formato especifico. Como recomendación tenemos el siguiente formato:

### reporte_fecha_SFUdeA.xlsx

- Al inicio del archivo indica un nombre que permita saber que es el reporte del encuentro y un guión bajo ( _ ).
- Seguido, ubica la fecha del encuentro, iniciando por el mes, día y año, todo separando por guiones bajos ( _ ).
- Por último, las iniciales de la estrategia, en este caso SFUdeA.
- Ejemplo: Para el archivo de Reporte del encuentro del 9 de Agosto de 2050 el archivo resultante seria el: reporte_08_09_2050_SFUdeA.xlsx (Recuerda que el .xslx es el tipo de archivo, por lo que no lo debes cambiar)

\
Ahora ya puedes correr el codigo, el archivo final se encontrará en el panel de archivos con el nombre asignado. Puede que dicho archivo tarde un poco en aparecer, en caso de no verlo rápido, puedes dar clic en el boton de recargar los archivos.

In [ ]:
# Esteban Motta Ruiz - Auxiliar de Porgramación en Ude@
# Edición de informes de Zoom - 2 de Junio de 2022
# Informes de Somos Familia UdeA
# esteban.motta@udea.edu.co


direccion_reg <- "/content/reg_08_06_2023_SFUdeA.csv" # Debes editar esta sección
direccion_par <- "/content/par_08_06_2023_SFUdeA.csv" # Debes editar esta sección

nombre_arch <- "/content/Reporte_08_06_2023_SFUdeA.xlsx" # Debes editar esta sección

Editar_Reporte_SFUdeA <- function(d1, d2, nom) {

    dir1 <- read.csv(d1, encoding = "UTF-8")
    dir2 <- read.csv(d2, encoding = "UTF-8")

    library(stringr)

    for (i in 1:length(dir1)) {
        dir1[i] <- str_replace_all(dir1[, i], " ", " ")
    }

    for (i in 1:length(dir2)) {
        dir2[i] <- str_replace_all(dir2[, i], " ", " ")
    }

    asist <- rep(1, times = length(dir1[, 1]))
    ins <- subset.data.frame(dir1[, c(1, 2, 3, 6, 7, 8, 9, 10)])
    ins_dt <- cbind(asist, ins)

    colnom1 <- c("Asist", "Nombre", "Apellido", "Correo", "Vinculacion",
                 "Unidad", "Familia", "Niños", "Tema")
    colnames(ins_dt) <- colnom1

    ins_dt$Correo <- tolower(ins_dt$Correo)

    ast <- subset.data.frame(dir2[, c(1, 2)])

    colnom2 <- c("Nombre", "Correo")
    colnames(ast) <- colnom2

    ast_dt <- merge(x = ins_dt,
                    y = ast,
                    by = "Correo")

    ins_vin <- aggregate(Asist ~ Vinculacion,
                         data = ins_dt,
                         FUN = sum)
    ins_uni <- aggregate(Asist ~ Unidad,
                         data = ins_dt,
                         FUN = sum)

    ast_vin <- aggregate(Asist ~ Vinculacion,
                         data = ast_dt,
                         FUN = sum)
    ast_uni <- aggregate(Asist ~ Unidad,
                         data = ast_dt,
                         FUN = sum)

    ast_dt2 <- ast_dt[, 1:9]
    colnames(ast_dt2)[3] <- "Nombre"
    ast_dt2$Asist <- "1"

    no_ast <- data.frame(Correo = setdiff(ins_dt$Correo, ast_dt2$Correo))
    no_ast_dt <- merge(x = ins_dt,
                       y = no_ast,
                       by.x = "Correo",
                       by.y = "Correo")

    no_ast_dt$Asist <- "0"

    general <- rbind(ast_dt2, no_ast_dt)
    general <- general[c(2, 3, 4, 1, 5, 6, 7, 8, 9)]
    general$Asist <- as.numeric(general$Asist)

    ninos <- data.frame(table(general$Niños))
    colnames(ninos) <- c("Respuesta", "Cantidad")

    familia <- data.frame(table(ins_dt$Familia))
    colnames(familia) <- c("Respuesta", "Cantidad")

    library(openxlsx)

    archivo_final <- list("Reporte General" = general,
                          "Ins Vinculacion" = ins_vin,
                          "Ins Unidad" = ins_uni,
                          "Ast Vinculacion" = ast_vin,
                          "Ast Unidad" = ast_uni,
                          "Familia" = familia,
                          "Niños" = ninos)


    write.xlsx(archivo_final,
               nom,
               colNames = TRUE,
               rowNames = FALSE)

}

Editar_Reporte_SFUdeA(d1 = direccion_reg,
                      d2 = direccion_par,
                      nom = nombre_arch)